In [1]:
import math
import numpy as np
import chainer
from chainer import backend
from chainer import backends
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, initializers, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
import random
import cupy

In [2]:
mempool = cupy.get_default_memory_pool()
pinned_mempool = cupy.get_default_pinned_memory_pool()

In [3]:
print(mempool.used_bytes())  
print(mempool.total_bytes()) 
print(pinned_mempool.n_free_blocks())

0
0
0


In [19]:
## Cleaning GPU memory
mempool.free_all_blocks()
pinned_mempool.free_all_blocks()

### We need to append all the data and shuffle it. so it would scatter properly.

Using the cleaned data which store in advance.

In [4]:
f = open('Data1.txt', 'r')
x1 = f.readlines()
f.close()

In [5]:
f = open('Data2.txt', 'r')
x2 = f.readlines()
f.close()

In [6]:
f = open('Data3.txt', 'r')
x3 = f.readlines()
f.close()

In [7]:
totalx = x1+x2+x3
random.shuffle(totalx)
x = totalx

### X now become a list of string as we read from txt file.
we need to convert the string into list.

[1:29999] is all the input x

[30001:30003] is the score. or or [30001:30004] if it is 100(as all scores is between 80-100)

In [8]:
import time

data1_fin = []
start = time.time()
for index in range(10000):
    temp_data = x[index][1:29999]
    if x[index][30003] == '0': #the score is 100
        score = int(x[index][30001:30004])
    else: #the score is between 80-99
        score = int(x[index][30001:30003])
    indx = int(x[index][30005:-2])
    temp_data1 = temp_data.split(', ')
    data = [int(i) for i in temp_data1]
    data.append(score)
    data.append(indx)
    data1_fin.append(data)

end=time.time()
print(end-start)

14.524568557739258


## Setting up train and test data

In [9]:
x = list()
y = list()

for i in range(len(data1_fin)):
    x.append(data1_fin[i][0:-2])
    y.append([data1_fin[i][-2]])

In [10]:
x1 = cuda.cupy.array(x, dtype=cupy.float32)
x2 = np.array(x, dtype=np.float32)

In [16]:
y1 = cuda.cupy.array(y, dtype=cupy.float32)
y2 = np.array(y, dtype=np.float32)

In [18]:
print(len(y2[0]))

1


In [19]:
number_of_data = len(x)
# variance_of_noise = 0.12
train_ratio = 0.8 #how much data will be used in training
slice_position = int(number_of_data * train_ratio)
x_train = x1[:slice_position]
y_train = y1[:slice_position]
x_train = Variable(x_train)
y_train = Variable(y_train)

x_test = x1[slice_position:]
y_test = y1[slice_position:]
x_test = Variable(x_test)
y_test = Variable(y_test)

In [20]:
x_train2 = x2[:slice_position]
y_train2 = y2[:slice_position]
x_train2 = Variable(x_train2)
y_train2 = Variable(y_train2)

x_test2 = x2[slice_position:]
y_test2 = y2[slice_position:]
x_test2 = Variable(x_test2)
y_test2 = Variable(y_test2)

In [22]:
import numpy as np
import matplotlib.pyplot as plt
from chainer import Chain, Variable,optimizers
import chainer.functions as F
import chainer.links as L

start = time.time()
#### Defining a class for our NN model
class Regression(Chain):
    def __init__(self):
        super(Regression,self).__init__(
            ### One hidden layer
            l1 = L.Linear(10000,4000),
            l2 = L.Linear(4000,1),
    )

    def __call__(self,x):
        ### We define non-linear function here
        ### We use relu but I suggest to use sigmoid just because I like it!
        h = F.relu(self.l1(x)) 
        h = self.l2(h)
        return h

if __name__ == "__main__":
    #Parameters
    batchsize = 100
    max_epoch = 10
        
    model = Regression()
    model.to_gpu(0)
    optimizer = optimizers.MomentumSGD(lr=0.01, momentum=0.9)
    optimizer.setup(model)

    #Train
    N = len(x_train)
    perm = cupy.random.permutation(N)
    for epoch in range(max_epoch):
        for i in range(0,N,batchsize):
            x_train_batch = x_train[perm[i:i + batchsize]]
            y_train_batch = y_train[perm[i:i + batchsize]]

            ## Before calculating GD, we need to clear the gradients.
            model.cleargrads()
            loss = F.mean_squared_error(model(x_train_batch),y_train_batch)
            ### I beleive this one below calculate backward, which means calculating gradient for each parameter
            loss.backward()
            ### Now, using the gradient calculated above, they update the parameter
            optimizer.update()
            
        print("epoch:",epoch,"loss:",loss.data)

#Test
    model.cleargrads()
    t_test = model(x_test)
    loss = F.mean_squared_error(t_test,y_test)
    print("loss of test data:",loss.data)

    #x_grandtruth = np.arange(-1.7,1.7,0.1)
    #y_grandtruth = x_grandtruth**4-2*x_grandtruth**2+0.5*x_grandtruth+2.0

    #Plot1
    # plt.plot(x_grandtruth,y_grandtruth,color="red",label="Grand Truth")
    
end=time.time()
print("used time: ", end-start)

epoch: 0 loss: 12.182865
epoch: 1 loss: 10.881067
epoch: 2 loss: 5.102632
epoch: 3 loss: 3.8800995
epoch: 4 loss: 3.3558893
epoch: 5 loss: 3.0711718
epoch: 6 loss: 2.8516386
epoch: 7 loss: 2.713988
epoch: 8 loss: 2.568303
epoch: 9 loss: 2.4890976
loss of test data: 4.3794103
used time:  17.14109468460083


In [21]:
start = time.time()
#### Defining a class for our NN model
class Regression(Chain):
    def __init__(self):
        super(Regression,self).__init__(
            ### One hidden layer
            l1 = L.Linear(10000,4000),
            l2 = L.Linear(4000,1),
    )

    def __call__(self,x):
        ### We define non-linear function here
        ### We use relu but I suggest to use sigmoid just because I like it!
        h = F.relu(self.l1(x))
        h = self.l2(h)
        return h

if __name__ == "__main__":
    #Parameters
    batchsize = 100
    max_epoch = 10
        
    model2 = Regression()
    optimizer = optimizers.MomentumSGD(lr=0.01, momentum=0.9)
    optimizer.setup(model2)

    #Train
    N = len(x_train2)
    perm = np.random.permutation(N)
    for epoch in range(max_epoch):
        for i in range(0,N,batchsize):
            x_train_batch2 = x_train2[perm[i:i + batchsize]]
            y_train_batch2 = y_train2[perm[i:i + batchsize]]

            ## Before calculating GD, we need to clear the gradients.
            model2.cleargrads()
            loss = F.mean_squared_error(model2(x_train_batch2),y_train_batch2)
            ### I beleive this one below calculate backward, which means calculating gradient for each parameter
            loss.backward()
            ### Now, using the gradient calculated above, they update the parameter
            optimizer.update()
            
        print("epoch:",epoch,"loss:",loss.data)

#Test
    model2.cleargrads()
    t_test2 = model2(x_test2)
    loss = F.mean_squared_error(t_test2,y_test2)
    print("loss of test data:",loss.data)

    #x_grandtruth = np.arange(-1.7,1.7,0.1)
    #y_grandtruth = x_grandtruth**4-2*x_grandtruth**2+0.5*x_grandtruth+2.0

    #Plot1
    # plt.plot(x_grandtruth,y_grandtruth,color="red",label="Grand Truth")
    
end=time.time()
print("used time: ", end-start)

epoch: 0 loss: 13.460535
epoch: 1 loss: 10.387961
epoch: 2 loss: 5.5507107
epoch: 3 loss: 3.5093808
epoch: 4 loss: 3.1015766
epoch: 5 loss: 2.9695795
epoch: 6 loss: 2.872941
epoch: 7 loss: 2.8172054
epoch: 8 loss: 2.7580001
epoch: 9 loss: 2.7018926
loss of test data: 3.967285
used time:  436.88626527786255


In [17]:
print('GPU availability:', chainer.cuda.available)
print('cuDNN availablility:', chainer.cuda.cudnn_enabled)

GPU availability: True
cuDNN availablility: True


In [ ]:
y_train_batch2[0]